In [2]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

#-------------------------------------
from datetime import datetime

In [3]:
df = pd.read_csv('./files/df_arabica_clean.csv')

In [4]:
df.head(3)

,Unnamed: 0,ID,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,Producer,Number of Bags,Bag Weight,In-Country Partner,Harvest Year,Grading Date,Owner,Variety,Status,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Expiration,Certification Body,Certification Address,Certification Contact
0,0,0,Colombia,Finca El Paraiso,CQU2022015,Finca El Paraiso,NaN,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,1,35 kg,Japan Coffee Exchange,2021 / 2022,"September 21st, 2022",Coffee Quality Union,Castillo,Completed,Double Anaerobic Washed,8.58,8.50,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
1,1,1,Taiwan,Royal Bean Geisha Estate,"The 2022 Pacific Rim Coffee Summit,T037",Royal Bean Geisha Estate,NaN,Taiwan Coffee Laboratory,1200,Chiayi,曾福森,1,80 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Completed,Washed / Wet,8.50,8.50,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
2,2,2,Laos,OKLAO coffee farms,"The 2022 Pacific Rim Coffee Summit,LA01",oklao coffee processing plant,NaN,Taiwan Coffee Laboratory,1300,Laos Borofen Plateau,WU TAO CHI,19,25 kg,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,"November 15th, 2022",Taiwan Coffee Laboratory 台灣咖啡研究室,Java,Completed,Semi Washed,8.33,8.42,8.08,8.17,7.92,8.17,10.0,10.0,10.0,8.33,0.0,87.42,10.4,0,0,yellowish,2,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"


In [5]:
df.columns

Index(['Unnamed: 0', 'ID', 'Country of Origin', 'Farm Name', 'Lot Number',
       'Mill', 'ICO Number', 'Company', 'Altitude', 'Region', 'Producer',
       'Number of Bags', 'Bag Weight', 'In-Country Partner', 'Harvest Year',
       'Grading Date', 'Owner', 'Variety', 'Status', 'Processing Method',
       'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance',
       'Uniformity', 'Clean Cup', 'Sweetness', 'Overall', 'Defects',
       'Total Cup Points', 'Moisture Percentage', 'Category One Defects',
       'Quakers', 'Color', 'Category Two Defects', 'Expiration',
       'Certification Body', 'Certification Address', 'Certification Contact'],
      dtype='object')

In [6]:
df_c = df.drop(columns=['Unnamed: 0','Lot Number','ICO Number','Number of Bags', 'Bag Weight','Grading Date','Status','Expiration','Certification Address', 'Certification Contact'])

In [7]:
df_c.head(10)

,ID,Country of Origin,Farm Name,Mill,Company,Altitude,Region,Producer,In-Country Partner,Harvest Year,Owner,Variety,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Certification Body
0,0,Colombia,Finca El Paraiso,Finca El Paraiso,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,Japan Coffee Exchange,2021 / 2022,Coffee Quality Union,Castillo,Double Anaerobic Washed,8.58,8.50,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,Japan Coffee Exchange
1,1,Taiwan,Royal Bean Geisha Estate,Royal Bean Geisha Estate,Taiwan Coffee Laboratory,1200,Chiayi,曾福森,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Washed / Wet,8.50,8.50,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,Taiwan Coffee Laboratory 台灣咖啡研究室
2,2,Laos,OKLAO coffee farms,oklao coffee processing plant,Taiwan Coffee Laboratory,1300,Laos Borofen Plateau,WU TAO CHI,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,Taiwan Coffee Laboratory 台灣咖啡研究室,Java,Semi Washed,8.33,8.42,8.08,8.17,7.92,8.17,10.0,10.0,10.0,8.33,0.0,87.42,10.4,0,0,yellowish,2,Taiwan Coffee Laboratory 台灣咖啡研究室
3,3,Costa Rica,La Cumbre,La Montana Tarrazu MIll,Coffee Quality Union,1900,"Los Santos,Tarrazu",Santa Maria de Dota,Japan Coffee Exchange,2022,Coffee Quality Union,Gesha,Washed / Wet,8.08,8.17,8.17,8.25,8.17,8.08,10.0,10.0,10.0,8.25,0.0,87.17,11.8,0,0,green,0,Japan Coffee Exchange
4,4,Colombia,Finca Santuario,Finca Santuario,Coffee Quality Union,1850-2100,"Popayan,Cauca",Camilo Merizalde,Japan Coffee Exchange,2022,Coffee Quality Union,Red Bourbon,"Honey,Mossto",8.33,8.33,8.08,8.25,7.92,7.92,10.0,10.0,10.0,8.25,0.0,87.08,11.6,0,2,yellow-green,2,Japan Coffee Exchange
5,5,Guatemala,La Colina,Dinámica Café,Taiwan Coffee Laboratory,1668,Chimaltenango,Emilio Antonio Medina Garcia,Taiwan Coffee Laboratory 台灣咖啡研究室,2022,Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Washed / Wet,8.33,8.33,8.25,7.83,7.83,8.17,10.0,10.0,10.0,8.25,0.0,87.00,10.7,0,0,green,2,Taiwan Coffee Laboratory 台灣咖啡研究室
6,6,Taiwan,野牡丹咖啡莊園 Melastoma Coffee Estate,野牡丹咖啡,Taiwan Coffee Laboratory,1250,Chiayi,黃保錫,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,Taiwan Coffee Laboratory 台灣咖啡研究室,Gesha,Washed / Wet,8.33,8.17,8.08,8.00,7.83,8.25,10.0,10.0,10.0,8.25,0.0,86.92,9.1,0,0,green,0,Taiwan Coffee Laboratory 台灣咖啡研究室
7,7,Taiwan,七彩琉璃咖啡莊園 Chi Tsai Liu Li Ecological Farm,七彩琉璃咖啡莊園,Taiwan Coffee Laboratory,1200,Chiayi,莊家榮,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,Taiwan Coffee Laboratory 台灣咖啡研究室,Sl34+Gesha,Natural / Dry,8.25,8.25,8.17,8.00,7.92,8.08,10.0,10.0,10.0,8.08,0.0,86.75,10.0,0,0,yellow green,1,Taiwan Coffee Laboratory 台灣咖啡研究室
8,8,Taiwan,亮軒咖啡莊園 Liang Xuan Coffee Farm,亮軒咖啡莊園,Taiwan Coffee Laboratory,1250,Chiayi,鍾旭亮,Taiwan Coffee Laboratory 台灣咖啡研究室,2021 / 2022,Taiwan Coffee Laboratory 台灣咖啡研究室,SL34,Washed / Wet,8.08,8.08,8.25,8.08,7.92,8.00,10.0,10.0,10.0,8.25,0.0,86.67,10.8,0,0,greenish,0,Taiwan Coffee Laboratory 台灣咖啡研究室
9,9,"Tanzania, United Republic Of",MASHIMA AMCOS,GOURMET COFFEE MILL,DORMAN (T) LIMITED,1400-1700,KILIMANJARO,DORMAN (T) LIMITED,Kenya Coffee Traders Association,2022 / 2023,Gilbert Maina,Bourbon,Washed / Wet,8.08,8.17,8.08,8.17,8.00,8.00,10.0,10.0,10.0,8.00,0.0,86.50,11.0,0,0,greenish,0,Kenya Coffee Traders Association


In [8]:
df_c.shape

(207, 31)

In [9]:
df['Farm Name'].unique()

array(['Finca El Paraiso', 'Royal Bean Geisha Estate',
       'OKLAO coffee farms', 'La Cumbre', 'Finca Santuario', 'La Colina',
       '野牡丹咖啡莊園 Melastoma Coffee Estate',
       '七彩琉璃咖啡莊園 Chi Tsai Liu Li Ecological Farm',
       '亮軒咖啡莊園 Liang Xuan Coffee Farm', 'MASHIMA AMCOS', 'TADE GG',
       'Karen Acajabon Coffee Farm', '古峰咖啡莊園 Goodfun Coffee Farm',
       'Gelana Geisha', 'La Gaitania', '青葉咖啡莊園', 'Halo Bariti Cooprative',
       '東璧咖啡莊園', '櫻桃果古坑咖啡莊園', 'BURKA', 'Finca Vista Hermosa',
       '仲大叔咖啡莊園Uncle Chung.s Coffee Farm', 'Hom Doi', 'El Diamante',
       '永舜咖啡莊園', 'Fazenda Recreio', '鄒築園ZouZhouYuan', '嵩岳咖啡莊園', '他扶芽有機農園',
       'Hokukano Ranch', '鄉庭有機農場Siang-Ting Organic Farm',
       '金讚咖啡農莊園(Jinzan Coffee Estate)', 'Eshetu farm',
       'Uraga Bisrat washing station', 'YHAENU PLC FARM', '卓武山咖啡農場',
       '御香咖啡園 YU SIANG Coffee Estate', 'Small Holder', '皇庭咖啡莊園',
       '花蓮縣秀林鄉特用作物（咖啡）產銷第一班Agriculture Production and Marketing Groups of Hualien Shlin township special crop (coff

In [10]:
patron_regex = '[^\\u4e00-\\u9fff]'

In [11]:
filtro = df_c[df_c['Farm Name'].str.contains(patron_regex, regex=True, na=False)]
filtro['Farm Name'].unique()

array(['Finca El Paraiso', 'Royal Bean Geisha Estate',
       'OKLAO coffee farms', 'La Cumbre', 'Finca Santuario', 'La Colina',
       '野牡丹咖啡莊園 Melastoma Coffee Estate',
       '七彩琉璃咖啡莊園 Chi Tsai Liu Li Ecological Farm',
       '亮軒咖啡莊園 Liang Xuan Coffee Farm', 'MASHIMA AMCOS', 'TADE GG',
       'Karen Acajabon Coffee Farm', '古峰咖啡莊園 Goodfun Coffee Farm',
       'Gelana Geisha', 'La Gaitania', 'Halo Bariti Cooprative', 'BURKA',
       'Finca Vista Hermosa', '仲大叔咖啡莊園Uncle Chung.s Coffee Farm',
       'Hom Doi', 'El Diamante', 'Fazenda Recreio', '鄒築園ZouZhouYuan',
       'Hokukano Ranch', '鄉庭有機農場Siang-Ting Organic Farm',
       '金讚咖啡農莊園(Jinzan Coffee Estate)', 'Eshetu farm',
       'Uraga Bisrat washing station', 'YHAENU PLC FARM',
       '御香咖啡園 YU SIANG Coffee Estate', 'Small Holder',
       '花蓮縣秀林鄉特用作物（咖啡）產銷第一班Agriculture Production and Marketing Groups of Hualien Shlin township special crop (coffee) 1st class',
       'Gideon Kule', '愛姬咖啡莊園iGfarm', 'varias fincas', 'PEÑA BLANCA',
      

In [12]:
#pip install pypinyin 
from pypinyin import lazy_pinyin

In [13]:
#función para limpiar caracteres chinos de grupos mixtos y pasarlos a alfabeto latino en grupos homogéneos
#grupo mixto: chino-latino
#grupo homogéneo: chino
def limpiar_car(nombre):
    # Caso 1: Mantener sin cambios (solo caracteres latinos y espacios)
    if pd.isna(nombre):
        return nombre
    
    if nombre.isascii():
        return nombre
    
    # Caso 2: Eliminar caracteres chinos
    if any('\u4e00' <= car <= '\u9fff' for car in nombre):
        # Elimina caracteres chinos del string
        return ''.join(car for car in nombre if not ('\u4e00' <= car <= '\u9fff'))
    
    # Caso 3: Convertir caracteres chinos a pinyin
    # Aquí mantenemos los caracteres no chinos y convertimos los chinos a pinyin
    return ''.join(lazy_pinyin(car) if '\u4e00' <= car <= '\u9fff' else car for car in nombre)

In [14]:
#probamos función
df_c['Farm Name'].apply(limpiar_car) #aplicar .unique()

0                          Finca El Paraiso
1                  Royal Bean Geisha Estate
2                        OKLAO coffee farms
3                                 La Cumbre
4                           Finca Santuario
                       ...                 
202                       Fazenda Conquista
203                        Finca San Felipe
204                                       -
205    Rosario de Maria II, Area de La Pila
206                           Walter Matter
Name: Farm Name, Length: 207, dtype: object

In [15]:
grupos_mixtos = df_c[['Farm Name', 'Mill', 'Company', 'Region', 'Producer', 'In-Country Partner','Owner','Certification Body']]

In [16]:
#aplicamos filtro a todas las columnas que comparten características 
for col in grupos_mixtos:
    df_c[col] = df_c[col].apply(limpiar_car)
    df_c[col] = df_c[col].astype(str).str.strip()
    print(f'Estos son los valores únicos de la columna {col.upper()}\n: {df_c[col].unique()}\n')
    print('--------------------\n')

Estos son los valores únicos de la columna FARM NAME
: ['Finca El Paraiso' 'Royal Bean Geisha Estate' 'OKLAO coffee farms'
 'La Cumbre' 'Finca Santuario' 'La Colina' 'Melastoma Coffee Estate'
 'Chi Tsai Liu Li Ecological Farm' 'Liang Xuan Coffee Farm'
 'MASHIMA AMCOS' 'TADE GG' 'Karen Acajabon Coffee Farm'
 'Goodfun Coffee Farm' 'Gelana Geisha' 'La Gaitania' ''
 'Halo Bariti Cooprative' 'BURKA' 'Finca Vista Hermosa'
 'Uncle Chung.s Coffee Farm' 'Hom Doi' 'El Diamante' 'Fazenda Recreio'
 'ZouZhouYuan' 'Hokukano Ranch' 'Siang-Ting Organic Farm'
 '(Jinzan Coffee Estate)' 'Eshetu farm' 'Uraga Bisrat washing station'
 'YHAENU PLC FARM' 'YU SIANG Coffee Estate' 'Small Holder'
 '（）Agriculture Production and Marketing Groups of Hualien Shlin township special crop (coffee) 1st class'
 'Gideon Kule' 'iGfarm' 'varias fincas' 'PEÑA BLANCA'
 'san francisco cotzal' 'PT Indo CafCo North Sumatera' 'Ruya Yayu Farm'
 'IYENGA' 'Fazenda Guariroba' 'La Santa' 'Varias Fincas'
 'SAN RAFAEL URIAS VALDES' 'Gam

In [17]:
df_c.head()

,ID,Country of Origin,Farm Name,Mill,Company,Altitude,Region,Producer,In-Country Partner,Harvest Year,Owner,Variety,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Certification Body
0,0,Colombia,Finca El Paraiso,Finca El Paraiso,Coffee Quality Union,1700-1930,"Piendamo,Cauca",Diego Samuel Bermudez,Japan Coffee Exchange,2021 / 2022,Coffee Quality Union,Castillo,Double Anaerobic Washed,8.58,8.50,8.42,8.58,8.25,8.42,10.0,10.0,10.0,8.58,0.0,89.33,11.8,0,0,green,3,Japan Coffee Exchange
1,1,Taiwan,Royal Bean Geisha Estate,Royal Bean Geisha Estate,Taiwan Coffee Laboratory,1200,Chiayi,,Taiwan Coffee Laboratory,2021 / 2022,Taiwan Coffee Laboratory,Gesha,Washed / Wet,8.50,8.50,7.92,8.00,7.92,8.25,10.0,10.0,10.0,8.50,0.0,87.58,10.5,0,0,blue-green,0,Taiwan Coffee Laboratory
2,2,Laos,OKLAO coffee farms,oklao coffee processing plant,Taiwan Coffee Laboratory,1300,Laos Borofen Plateau,WU TAO CHI,Taiwan Coffee Laboratory,2021 / 2022,Taiwan Coffee Laboratory,Java,Semi Washed,8.33,8.42,8.08,8.17,7.92,8.17,10.0,10.0,10.0,8.33,0.0,87.42,10.4,0,0,yellowish,2,Taiwan Coffee Laboratory
3,3,Costa Rica,La Cumbre,La Montana Tarrazu MIll,Coffee Quality Union,1900,"Los Santos,Tarrazu",Santa Maria de Dota,Japan Coffee Exchange,2022,Coffee Quality Union,Gesha,Washed / Wet,8.08,8.17,8.17,8.25,8.17,8.08,10.0,10.0,10.0,8.25,0.0,87.17,11.8,0,0,green,0,Japan Coffee Exchange
4,4,Colombia,Finca Santuario,Finca Santuario,Coffee Quality Union,1850-2100,"Popayan,Cauca",Camilo Merizalde,Japan Coffee Exchange,2022,Coffee Quality Union,Red Bourbon,"Honey,Mossto",8.33,8.33,8.08,8.25,7.92,7.92,10.0,10.0,10.0,8.25,0.0,87.08,11.6,0,2,yellow-green,2,Japan Coffee Exchange


In [18]:
df_c.isna().sum()

ID                      0
Country of Origin       0
Farm Name               0
Mill                    0
Company                 0
Altitude                1
Region                  0
Producer                0
In-Country Partner      0
Harvest Year            0
Owner                   0
Variety                 6
Processing Method       5
Aroma                   0
Flavor                  0
Aftertaste              0
Acidity                 0
Body                    0
Balance                 0
Uniformity              0
Clean Cup               0
Sweetness               0
Overall                 0
Defects                 0
Total Cup Points        0
Moisture Percentage     0
Category One Defects    0
Quakers                 0
Color                   0
Category Two Defects    0
Certification Body      0
dtype: int64

In [19]:
df_c.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,207.0,103.000000,59.899917,0.00,51.50,103.00,154.500,206.00
Aroma,207.0,7.721063,0.287626,6.50,7.58,7.67,7.920,8.58
Flavor,207.0,7.744734,0.279613,6.75,7.58,7.75,7.920,8.50
Aftertaste,207.0,7.599758,0.275911,6.67,7.42,7.58,7.750,8.42
Acidity,207.0,7.690290,0.259510,6.83,7.50,7.67,7.875,8.58
Body,207.0,7.640918,0.233499,6.83,7.50,7.67,7.750,8.25
Balance,207.0,7.644058,0.256299,6.67,7.50,7.67,7.790,8.42
Uniformity,207.0,9.990338,0.103306,8.67,10.00,10.00,10.000,10.00
Clean Cup,207.0,10.000000,0.000000,10.00,10.00,10.00,10.000,10.00
Sweetness,207.0,10.000000,0.000000,10.00,10.00,10.00,10.000,10.00


In [20]:
df_c.describe(include='object').T

,count,unique,top,freq
Country of Origin,207,22,Taiwan,61
Farm Name,207,134,,43
Mill,207,124,,40
Company,207,71,Taiwan Coffee Laboratory,51
Altitude,206,97,1200,23
Region,207,116,,27
Producer,207,124,,52
In-Country Partner,207,21,Taiwan Coffee Laboratory,83
Harvest Year,207,7,2021 / 2022,99
Owner,207,78,Taiwan Coffee Laboratory,30


In [21]:
df_c.columns

Index(['ID', 'Country of Origin', 'Farm Name', 'Mill', 'Company', 'Altitude',
       'Region', 'Producer', 'In-Country Partner', 'Harvest Year', 'Owner',
       'Variety', 'Processing Method', 'Aroma', 'Flavor', 'Aftertaste',
       'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean Cup', 'Sweetness',
       'Overall', 'Defects', 'Total Cup Points', 'Moisture Percentage',
       'Category One Defects', 'Quakers', 'Color', 'Category Two Defects',
       'Certification Body'],
      dtype='object')

In [22]:
nulos_cat = df_c[df_c.isnull().any(axis=1)]
nulos_cat


,ID,Country of Origin,Farm Name,Mill,Company,Altitude,Region,Producer,In-Country Partner,Harvest Year,Owner,Variety,Processing Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Overall,Defects,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Certification Body
23,23,Colombia,El Diamante,El Diamante,Coffee Quality Union,1350,Quindio,Farm Bedoya Arango,Japan Coffee Exchange,2022,Coffee Quality Union,Castillo Paraguaycito,NaN,8.08,8.00,7.83,8.17,7.75,7.83,10.0,10.0,10.0,8.00,0.0,85.67,11.3,0,0,brownish,2,Japan Coffee Exchange
25,25,Brazil,Fazenda Recreio,Dry mill,"WIT wealth, investments & trust",1250,Região Vulcânica,Diogo T. Dias de Macedo,Brazil Specialty Coffee Association,2022,Andre Clark,NaN,Pulped natural / honey,7.83,8.08,7.83,7.92,8.00,7.83,10.0,10.0,10.0,8.00,0.0,85.50,11.3,2,0,green,3,Brazil Specialty Coffee Association
44,44,Taiwan,iGfarm,iGfarm,Taiwan Coffee Laboratory,150,Shoufeng Township,LIU MENG TSUNG,Taiwan Coffee Laboratory,2021 / 2022,Liu Meng Tsung,Typica,NaN,7.92,8.00,7.92,8.00,7.58,7.75,10.0,10.0,10.0,7.83,0.0,85.00,10.9,0,0,green,0,Taiwan Coffee Laboratory
51,51,Indonesia,PT Indo CafCo North Sumatera,PT Indo CafCo,InterAmerican Coffee,1200,Aceh Tengah,PT Indo CafCo North Sumatera,NKG Quality Service (a division of Bernhard Ro...,2022,Charles Umeano,Catimor,NaN,7.83,7.92,7.75,7.83,7.83,7.83,10.0,10.0,10.0,7.83,0.0,84.83,11.9,0,3,bluish-green,2,NKG Quality Service (a division of Bernhard Ro...
97,97,Colombia,nan,"Racafé & CIA S.C.A, Km 5 Vía Pereira",Marubeni Corporation,1411,Pereira,RACAFE & CIA SCA,Japan Coffee Exchange,2022 / 2023,Ikuto Uehara,NaN,Washed / Wet,7.67,7.75,7.67,7.75,7.67,7.67,10.0,10.0,10.0,7.67,0.0,83.83,11.9,0,1,green,1,Japan Coffee Exchange
105,105,Colombia,nan,nan,Coffee Quality Institute,NaN,nan,nan,Barista and Coffee Academy of Asia,2022,Coffee Quality Institute,NaN,NaN,7.83,7.75,7.50,7.58,7.67,7.67,10.0,10.0,10.0,7.67,0.0,83.67,12.4,1,0,greenish,9,Barista and Coffee Academy of Asia
143,143,Taiwan,,yes,,1200,"Shibi, Gukeng Township, Yunlin County , Taiwa...",,Blossom Valley International,2022,GuoLiangZhi,Gesha,NaN,7.67,7.50,7.42,7.75,7.42,7.58,10.0,10.0,10.0,7.58,0.0,82.92,10.2,0,0,yellow-green,0,Blossom Valley International
177,177,Guatemala,Not Specificated,Dry Mill of Exportcafe,Marubeni Corporation,1350-1550,Oriente,"Exportcafe, S.A.",Japan Coffee Exchange,2021 / 2022,Ikuto Uehara,NaN,Washed / Wet,7.42,7.42,7.42,7.33,7.50,7.42,10.0,10.0,10.0,7.50,0.0,82.00,11.5,0,1,blue-green,2,Japan Coffee Exchange
178,178,Indonesia,Wahana Graha Makmur,PT. Wahana Graha Makmur Medan,InterAmerican Coffee,1350,Sumatra,PT. Wahana Graha Makmur Group,NKG Quality Service (a division of Bernhard Ro...,2022,Charles Umeano,NaN,Pulped natural / honey,7.67,7.33,7.25,7.67,7.42,7.25,10.0,10.0,10.0,7.42,0.0,82.00,12.5,0,0,bluish-green,0,NKG Quality Service (a division of Bernhard Ro...
194,194,Brazil,Various farm,Dry Mill of Cocapil,Marubeni Corporation,1200,Alta Mogiana-Ibiraci,Cocapil,Japan Coffee Exchange,2022 / 2023,Ikuto Uehara,NaN,Natural / Dry,7.42,7.42,7.25,7.25,7.17,7.25,10.0,10.0,10.0,7.25,0.0,81.00,10.6,0,2,greenish,1,Japan Coffee Exchange


In [23]:
df_c['Farm Name'].unique()
#eliminar espacios antes de títulos
#eliminar paréntesis 
#revisar: '\t\t', 'N / A', '-'


array(['Finca El Paraiso', 'Royal Bean Geisha Estate',
       'OKLAO coffee farms', 'La Cumbre', 'Finca Santuario', 'La Colina',
       'Melastoma Coffee Estate', 'Chi Tsai Liu Li Ecological Farm',
       'Liang Xuan Coffee Farm', 'MASHIMA AMCOS', 'TADE GG',
       'Karen Acajabon Coffee Farm', 'Goodfun Coffee Farm',
       'Gelana Geisha', 'La Gaitania', '', 'Halo Bariti Cooprative',
       'BURKA', 'Finca Vista Hermosa', 'Uncle Chung.s Coffee Farm',
       'Hom Doi', 'El Diamante', 'Fazenda Recreio', 'ZouZhouYuan',
       'Hokukano Ranch', 'Siang-Ting Organic Farm',
       '(Jinzan Coffee Estate)', 'Eshetu farm',
       'Uraga Bisrat washing station', 'YHAENU PLC FARM',
       'YU SIANG Coffee Estate', 'Small Holder',
       '（）Agriculture Production and Marketing Groups of Hualien Shlin township special crop (coffee) 1st class',
       'Gideon Kule', 'iGfarm', 'varias fincas', 'PEÑA BLANCA',
       'san francisco cotzal', 'PT Indo CafCo North Sumatera',
       'Ruya Yayu Farm', 'IYE